<h1 align="center"><b>CS116.M12.KHCL - LẬP TRÌNH PYTHON CHO MÁY HỌC</b></h1>
<h1 align="center"><b>Assignment 11: XGBoost</b></h1>


**Sinh viên thực hiện:**
- Thái Trần Khánh Nguyên - 19520188

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import xgboost as xgb

# Exercise 1 + 2: 

In [ ]:
data = pd.read_csv('Data.csv')
data.head()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [ ]:
data.isnull().values.any()

False

In [ ]:
# Cột Sample code number không ảnh hưởng đến kết quả nên sẽ bị bỏ đi
X, y = data.iloc[:,1:-1], data['Class']
X = np.array(X)

In [ ]:
kf = KFold(n_splits=5, random_state=19520188, shuffle=True)

## Gradient Boosting (GBM)

In [ ]:
accuracy_list= []
for train_index, test_index in kf.split(X):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)

  model = GradientBoostingClassifier(learning_rate=0.01,random_state=1)
  model.fit(X_train, y_train)
  accuracy = model.score(X_test, y_test)
  accuracy_list.append(accuracy)
  print("Accuracy: {:.2f}%".format(accuracy*100))

print('---------------------------------------------')
print("Mean Accuracy: {:.2f}% +- {:.2f}".format((np.mean(accuracy_list)*100), np.std(accuracy_list)*100))

Accuracy: 94.16%
Accuracy: 93.43%
Accuracy: 94.16%
Accuracy: 96.32%
Accuracy: 94.85%
---------------------------------------------
Mean Accuracy: 94.59% +- 0.98


## Extreme Gradient Boosting (XGBoost)

In [ ]:
accuracy_list= []
for train_index, test_index in kf.split(X):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)

  model = xgb.XGBClassifier(random_state=1, eta=0.01)
  model.fit(X_train, y_train)
  accuracy = model.score(X_test, y_test)
  accuracy_list.append(accuracy)
  print("Accuracy: {:.2f}%".format(accuracy*100))

print('---------------------------------------------')
print("Mean Accuracy: {:.2f}% +- {:.2f}".format((np.mean(accuracy_list)*100), np.std(accuracy_list)*100))

Accuracy: 97.08%
Accuracy: 97.08%
Accuracy: 97.08%
Accuracy: 97.79%
Accuracy: 97.06%
---------------------------------------------
Mean Accuracy: 97.22% +- 0.29


# Exercise 3:

In [ ]:
data = pd.read_csv('loan_data.csv')
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [ ]:
data.isnull().values.any()

True

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y


In [ ]:
#Chuẩn hóa các feature có dạng Binary về 1-0
data['Gender'] = data['Gender'].apply(lambda x : 1 if x=='Male' else 0)
data['Married'] = data['Married'].apply(lambda x : 1 if x=='Yes' else 0)
data['Education'] = data['Education'].apply(lambda x : 1 if x=='Graduate' else 0)
data['Self_Employed'] = data['Self_Employed'].apply(lambda x : 1 if x=='Yes' else 0)
data['Dependents'] = data['Dependents'].apply(lambda x : 3 if x=='3+' else x)
data['Loan_Status'] = data['Loan_Status'].apply(lambda x : 1 if x=='Y' else 0)

#One-hot vector
data = pd.concat([data, pd.get_dummies(data['Property_Area'])], axis=1)

#Loại bỏ các feature không cần thiết
data = data.drop(columns=['Loan_ID', 'Property_Area'])
data.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Rural,Semiurban,Urban
1,1,1,1,1,0,4583,1508.0,128.0,360.0,1.0,0,1,0,0
2,1,1,0,1,1,3000,0.0,66.0,360.0,1.0,1,0,0,1
3,1,1,0,0,0,2583,2358.0,120.0,360.0,1.0,1,0,0,1
4,1,0,0,1,0,6000,0.0,141.0,360.0,1.0,1,0,0,1
5,1,1,2,1,1,5417,4196.0,267.0,360.0,1.0,1,0,0,1


In [ ]:
X, y =np.array(data.drop(columns=['Loan_Status'])), np.array(data['Loan_Status'])

In [ ]:
accuracy_list= []
for train_index, test_index in kf.split(X):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)

  model = GradientBoostingClassifier(learning_rate=0.01,random_state=1)
  model.fit(X_train, y_train)
  accuracy = model.score(X_test, y_test)
  accuracy_list.append(accuracy)
  print("Accuracy: {:.2f}%".format(accuracy*100))

print('---------------------------------------------')
print("Mean Accuracy: {:.2f}% +- {:.2f}".format((np.mean(accuracy_list)*100), np.std(accuracy_list)*100))

Accuracy: 82.29%
Accuracy: 75.00%
Accuracy: 80.21%
Accuracy: 78.12%
Accuracy: 84.38%
---------------------------------------------
Mean Accuracy: 80.00% +- 3.25


In [ ]:
accuracy_list= []
for train_index, test_index in kf.split(X):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)

  model = xgb.XGBClassifier(random_state=1, eta=0.01)
  model.fit(X_train, y_train)
  accuracy = model.score(X_test, y_test)
  accuracy_list.append(accuracy)
  print("Accuracy: {:.2f}%".format(accuracy*100))

print('---------------------------------------------')
print("Mean Accuracy: {:.2f}% +- {:.2f}".format((np.mean(accuracy_list)*100), np.std(accuracy_list)*100))

Accuracy: 80.21%
Accuracy: 79.17%
Accuracy: 77.08%
Accuracy: 71.88%
Accuracy: 82.29%
---------------------------------------------
Mean Accuracy: 78.12% +- 3.55
